In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from tqdm import tqdm

2024-07-28 10:46:29.106517: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-28 10:46:29.257470: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 10:46:29.257526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 10:46:29.276006: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 10:46:29.318643: I tensorflow/core/platform/cpu_feature_guar

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy

In [11]:
class VGGNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()
        
        self.add(Conv2D(
            64,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu",
            input_shape=input_shape
        ))

        self.add(Conv2D(
            64,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Flatten()),

        self.add(Dense(4096, activation="relu"))
        self.add(Dense(4096, activation="relu"))
        self.add(Dense(1000, activation="softmax"))

        self.compile(
            optimizer=Adam(0.001),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

In [12]:
num_epochs = 10
batch_size = 32
Input_shape = (224, 224, 3)
num_classes = 3


model = VGGNet(Input_shape, num_classes=num_classes)

model.summary()

Model: "vgg_net_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_16 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 112, 112, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_18 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                    